In [1]:
PROJECT_ID = "boxwood-well-386122"
REGION = "us-central1"
BUCKET_URI = "gs://vijay-lens-feature-store-temp"  
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

In [2]:
# Credentials saved to file: [/Users/vijay/.config/gcloud/application_default_credentials.json]
# These credentials will be used by any library that requests Application Default Credentials (ADC).

! gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=rdpdpaK87ASxK62DrhneRmumc7CyF3&access_type=offline&code_challenge=pLWQ3MXtwSZGNhQneiwaw_ML_8ynu6sp5wO9960RGUA&code_challenge_method=S256


Credentials saved to file: [/Users/vijay/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "boxwood-well-386122" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the

In [3]:
# Create bucket if not already created
# ! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

In [3]:
from google.cloud.aiplatform import Feature, Featurestore

In [10]:
FEATURESTORE_ID = "lens_featurestore_t1"
# ONLINE_STORE_FIXED_NODE_COUNT = None
# fs = Featurestore.create(
#     featurestore_id=FEATURESTORE_ID,
#     online_store_fixed_node_count=ONLINE_STORE_FIXED_NODE_COUNT,
#     project=PROJECT_ID,
#     location=REGION,
#     sync=True,
# )

In [11]:
fs = Featurestore(
    featurestore_name=FEATURESTORE_ID
)
print(fs.gca_resource)

name: "projects/1181216607/locations/us-central1/featurestores/lens_featurestore_t1"
create_time {
  seconds: 1684940132
  nanos: 189557000
}
update_time {
  seconds: 1684940132
  nanos: 321577000
}
etag: "AMEw9yMJVzrc3LOG2jZ4jSQTscq8flErGxxEVZakE6HtehswzTpu_st4WFHk8mEQsDmB"
online_serving_config {
}
state: STABLE



In [13]:
# Create the 'profiles' entity type
profiles_entity_type = fs.create_entity_type(
    entity_type_id="profiles",
    description="Lens Profile entity",
)

Creating EntityType
Create EntityType backing LRO: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_t1/entityTypes/profiles/operations/4154133256779857920
EntityType created. Resource name: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_t1/entityTypes/profiles
To use this EntityType in another session:
entity_type = aiplatform.EntityType('projects/1181216607/locations/us-central1/featurestores/lens_featurestore_t1/entityTypes/profiles')


In [14]:
# Create the "posts" entity type
posts_entity_type = fs.create_entity_type(
    entity_type_id="posts",
    description="Lens Post entity",
)

Creating EntityType
Create EntityType backing LRO: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_t1/entityTypes/posts/operations/2244607014774767616
EntityType created. Resource name: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_t1/entityTypes/posts
To use this EntityType in another session:
entity_type = aiplatform.EntityType('projects/1181216607/locations/us-central1/featurestores/lens_featurestore_t1/entityTypes/posts')


In [5]:
fs.list_entity_types()

 resource name: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_d2/entityTypes/posts,
 resource name: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_d2/entityTypes/profiles]

In [15]:
profiles_entity_type = fs.get_entity_type(entity_type_id="profiles")
posts_entity_type = fs.get_entity_type(entity_type_id="posts")
print(profiles_entity_type)
print(posts_entity_type)

resource name: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_t1/entityTypes/profiles
resource name: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_t1/entityTypes/posts


In [16]:
POSTS_FEATURE_CONFIGS = {
        "age": {"value_type": "INT64", "description": "Age of the post"},
        "mirrors": {"value_type": "INT64", "description": "Number of mirrors"},
        "collects": {"value_type": "INT64", "description": "Number of collects"},
        "comments": {"value_type": "INT64", "description": "Number of comments"},
        "upvotes": {"value_type": "INT64", "description": "Number of upvotes"},
        "downvotes": {"value_type": "INT64", "description": "Number of downvotes"},
        "is_original": {
            "value_type": "BOOL", 
            "description": "Original post or mirror of another post"
        },
        "language": {"value_type": "STRING", "description": ""},
        "region": {"value_type": "STRING", "description": "Locale"},
        "is_content_warning": {"value_type": "BOOL", "description": ""},
        "content_warning": {"value_type": "STRING", "description": ""},
        "main_content_focus": {
            "value_type": "STRING", 
            "description": "IMAGE, VIDEO or TEXT_ONLY"
        },
        "custom_filters_gardener_flagged": {"value_type": "BOOL", "description": ""},
        "recommend": {
             "value_type": "STRING",
             "description": "Label whether this post should be recommended or not",
        },
    }

posts_entity_type.batch_create_features(feature_configs=POSTS_FEATURE_CONFIGS, sync=True)

Batch creating features EntityType entityType: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_t1/entityTypes/posts
Batch create Features EntityType entityType backing LRO: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_t1/entityTypes/posts/operations/2363952404900085760
EntityType entityType Batch created features. Resource name: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_t1/entityTypes/posts


resource name: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_t1/entityTypes/posts

In [17]:
posts_feature_post_id = posts_entity_type.create_feature(
    feature_id="profile_id",
    value_type="STRING",
    description="Profile Id of the author of this post",
)

Creating Feature
Create Feature backing LRO: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_t1/entityTypes/posts/features/profile_id/operations/6311357478290325504
Feature created. Resource name: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_t1/entityTypes/posts/features/profile_id
To use this Feature in another session:
feature = aiplatform.Feature('projects/1181216607/locations/us-central1/featurestores/lens_featurestore_t1/entityTypes/posts/features/profile_id')


In [7]:
posts_entity_type.list_features()

 resource name: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_d2/entityTypes/posts/features/main_content_focus,
 resource name: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_d2/entityTypes/posts/features/comments,
 resource name: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_d2/entityTypes/posts/features/language,
 resource name: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_d2/entityTypes/posts/features/recommend,
 resource name: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_d2/entityTypes/posts/features/downvotes,
 resource name: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_d2/entityTypes/posts/features/mirrors,
 resource name: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_d2/entityTypes/posts/features/is_content_warning,
 resource name: projects/1181216607/locations/us-central1/featurestor

In [18]:
PROFILES_FEATURE_CONFIGS = {
    "followship_score": {
        "value_type": "DOUBLE", 
        "description": 
          """Global EigenTrust score computed using followship strategy"""
    },
    "followship_rank": {
        "value_type": "INT64", 
        "description": 
          """Global EigenTrust rank computed using followship strategy"""
    },
}

profiles_features = profiles_entity_type.batch_create_features(
    feature_configs=PROFILES_FEATURE_CONFIGS,
)

Batch creating features EntityType entityType: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_t1/entityTypes/profiles
Batch create Features EntityType entityType backing LRO: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_t1/entityTypes/profiles/operations/4741853008151707648
EntityType entityType Batch created features. Resource name: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_t1/entityTypes/profiles


In [ ]:
# MORE_PROFILES_FEATURE_CONFIGS = {
#     "profile_id": { "value_type": "STRING","description": "Profile Id from Lens" },
#     "age": { "value_type": "INT64", "description": "Age of the post" },
#     "posts": { "value_type": "INT64","description": "Number of posts by profile" },
#     "comments_in": { 
#         "value_type": "INT64", 
#         "description": "Number of comments this profile's posts have received" 
#     },
#     "comments_out": { 
#         "value_type": "INT64", 
#         "description": "Number of comments that this profile has made" 
#     },
#     "mirrors_in": { 
#         "value_type": "INT64", 
#         "description": "Number of mirrors this profile's posts have received" 
#     },
#     "mirrors_out": { 
#         "value_type": "INT64", 
#         "description": "Number of mirrors that this profile has made" 
#     },
#     "collects_in": { 
#         "value_type": "INT64", 
#         "description": "Number of collects this profile's posts have received" 
#     },
#     "collects_out": { 
#         "value_type": "INT64", 
#         "description": "Number of collects that this profile has made" 
#     },
#     "follows_in": { 
#         "value_type": "INT64", 
#         "description": "Number of profiles following this profile" 
#     },
#     "follows_out": { 
#         "value_type": "INT64", 
#         "description": "Number of profiles that this profile follows" 
#     },
# }

# profiles_features = profiles_entity_type.batch_create_features(
#     feature_configs=MORE_PROFILES_FEATURE_CONFIGS,
# )

In [11]:
profiles_entity_type.list_features()

 resource name: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_d2/entityTypes/profiles/features/followship_score,
 resource name: projects/1181216607/locations/us-central1/featurestores/lens_featurestore_d2/entityTypes/profiles/features/followship_rank]

## Sanity check featurestore

In [ ]:
my_features = Feature.search(query="featurestore_id={}".format(FEATURESTORE_ID))
my_features

In [ ]:
double_features = Feature.search(
    query="value_type=DOUBLE AND featurestore_id={}".format(FEATURESTORE_ID)
)
double_features[0].gca_resource